In [37]:
import json 
import pandas as pd
import re
import nltk
from nltk.util import ngrams
from collections import Counter

#Strategy 1 scan after the word won:
df = pd.read_json("gg2013.json/gg2013.json")
df2 = df[df['text'].str.contains(" won ") | df['text'].str.contains(" Won ")]
tweets = df2['text'].tolist()
keywords = []
regexp = re.compile(r'[!?.]+(?=$|\s)')

#Parse Keywords
for tweet in tweets:
    words = tweet.lower().split()
    index_won = words.index("won")
    if words[index_won + 1] == "best":
        keyword = "best "
        curr_index = index_won + 2
        while curr_index != len(words):
            if regexp.search(words[curr_index]) and len(words[curr_index]) > 2:
                keyword += words[curr_index]
                break
            keyword += words[curr_index] + " "
            curr_index += 1
        keywords.append(keyword)

#Analyze common phrase occurences
bigrams = []
trigrams = []
quadgrams = []
multigrams = []
for item in keywords:
    clean_string = re.sub('[^A-Za-z0-9]+', ' ', item)
    tokens = nltk.word_tokenize(clean_string)
    grams2 = ngrams(tokens, 2)
    grams3 = ngrams(tokens, 3)
    grams4 = ngrams(tokens, 4)
    grams9 = ngrams(tokens, 9)
    for gram in grams2:
        bigrams.append(gram)
    for gram in grams3:
        trigrams.append(gram)
    for gram in grams4:
        quadgrams.append(gram)
    for gram in grams9:
        multigrams.append(gram)

common2 = Counter(bigrams).most_common()
common3 = Counter(trigrams).most_common()
common4 = Counter(quadgrams).most_common()
common9 = Counter(multigrams).most_common()

common2 = filter(lambda x: x[0][0] == "best", common2)
common3 = filter(lambda x: x[0][0] == "best", common3)
common4 = filter(lambda x: x[0][0] == "best", common4)
common9 = filter(lambda x: x[0][0] == "best", common9)

common2 = list(common2)
common3 = list(common3)
common4 = list(common4)
common9 = list(common9)

combined = common2 + common3 + common4 + common9

combined

[(('best', 'director'), 76),
 (('best', 'actress'), 44),
 (('best', 'supporting'), 37),
 (('best', 'original'), 37),
 (('best', 'actor'), 32),
 (('best', 'picture'), 28),
 (('best', 'screenplay'), 18),
 (('best', 'motion'), 13),
 (('best', 'song'), 10),
 (('best', 'animated'), 10),
 (('best', 'tv'), 9),
 (('best', 'performance'), 8),
 (('best', 'drama'), 7),
 (('best', 'foreign'), 7),
 (('best', 'comedy'), 7),
 (('best', 'movie'), 7),
 (('best', 'dressed'), 6),
 (('best', 'pic'), 2),
 (('best', 'skin'), 1),
 (('best', 'blurry'), 1),
 (('best', 'life'), 1),
 (('best', 'show'), 1),
 (('best', 'anal'), 1),
 (('best', 'husband'), 1),
 (('best', 'film'), 1),
 (('best', 'flick'), 1),
 (('best', 'original', 'song'), 35),
 (('best', 'supporting', 'actress'), 21),
 (('best', 'director', 'at'), 18),
 (('best', 'director', 'for'), 13),
 (('best', 'motion', 'picture'), 13),
 (('best', 'actress', 'drama'), 13),
 (('best', 'supporting', 'actor'), 12),
 (('best', 'actor', 'in'), 11),
 (('best', 'actr